In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\kerry\AppData\Local\Temp\ipykernel_17724\1749505576.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



{"C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\li

In [2]:
#| label: fig-miracle-exponential
#| fig-cap: Growth of an Investment Account

import plotly.graph_objects as go
import numpy as np

trace1= go.Scatter(
    x=np.arange(31), 
    y=1.08**np.arange(31),
    mode="lines",
    name="8%"
)

trace2 = go.Scatter(
    x=np.arange(31), 
    y=1.04**np.arange(31),
    mode="lines",
    name="4%"
)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)
string = "year %{x}<br>balance = $%{y:.2f}"
fig.update_traces(hovertemplate=string)
fig.update_layout(
    template="none",
    xaxis_title="Year",
    yaxis_title="Account Balance",
    yaxis_tickprefix="$", 
    yaxis_tickformat=",.0f",
    legend=dict(
        yanchor="top", 
        y=0.99, 
        xanchor="left", 
        x=0.01
        )
)
fig.show()

In [3]:
#| label: fig-miracle-pvfactors
#| fig-cap: Present Value Factors

import plotly.graph_objects as go
import numpy as np

trace1= go.Scatter(
    x=np.arange(31), 
    y=1.08**(-np.arange(31)),
    mode="lines",
    name="8%"
)
trace2 = go.Scatter(
    x=np.arange(31), 
    y=1.04**(-np.arange(31)),
    mode="lines",
    name="4%"
)
fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)
string = "year %{x}<br>PV factor = %{y:.1%}"
fig.update_traces(hovertemplate=string)
fig.update_layout(
    template="none",
    xaxis_title="Year",
    yaxis_title="PV Factor",
    yaxis_tickformat=".0%",
    legend=dict(
        yanchor="top", 
        y=0.99, 
        xanchor="right", 
        x=0.99
        )
)
fig.show()

In [4]:
#| label: fig-retirement
#| fig-cap: 'A retirement account with m=360 month of deposits, n= 720 total months, a monthly rate of return of r= 0.5%, an initial balance of B_0 =  $100,000, a monthly deposit growth rate of g=0.1%, an initial deposit of D_1 =  $1,000, and monthly withdrawals equal to W = $10,000.'

import numpy as np 
import plotly.graph_objects as go

m = 360
n = 720
r = 0.005
B0 = 100000
g = 0.001
D1 = 1000
W = 10000
D = D1 * (1 + g) ** np.arange(m)

B = np.zeros(n + 1)
B[0] = B0
for i in range(1, m):
    B[i] = (1 + r) * B[i - 1] + D[i - 1]
B[m] = (1 + r) * B[m - 1] + D[m - 1] - W
for i in range(m + 1, n):
    B[i] = (1 + r) * B[i - 1] - W
B[n] = (1 + r) * B[n - 1]

string = "month %{x}<br>balance=$%{y:,.0f}<extra></extra>"
trace = go.Scatter(
    x=np.arange(n+1),
    y=B,
    mode="lines",
    hovertemplate=string
)
fig = go.Figure(trace)
fig.update_layout(
    template="none",
    xaxis_title="Month",
    yaxis_title="Account Balance",
    yaxis_tickprefix="$", yaxis_tickformat=".2s"
)
fig.show()